In [1]:
from google.colab import userdata
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
import json

api_key = userdata.get('openai-key')
os.environ['API_KEY'] = api_key
API_KEY = api_key

llm = ChatOpenAI(openai_api_key=API_KEY)

GOOGLE_API_KEY = userdata.get('jerry-argolis-api-key')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# setup the gemini pro
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def print_message(aiMessage, messageOnly=True):
    print("RESPONSE: {}".format(aiMessage.content))
    if messageOnly == False:
        print(json.dumps(aiMessage.additional_kwargs, indent=2))
        print(json.dumps(aiMessage.response_metadata, indent=2))
        print(json.dumps(aiMessage.usage_metadata, indent=2))


In [4]:
from langchain.agents import tool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.schema import SystemMessage
from langchain.globals import set_debug
set_debug(False)

In [62]:
import requests
from requests.auth import HTTPBasicAuth

def google_map(activity: str, city: str = "부산") -> dict:
    url = f"http://34.64.181.15:8991/google_map"
    headers = {
        "Authentication": "application/json"
    }
    result = response = requests.get(url, params={"activity": activity, "city": city}, auth=HTTPBasicAuth("xxxxxxxx", "xxxxxxxx")).json()
    return result

google_map_tool = Tool(
    name="google_map",
    func=google_map,
    description="Get accommodations based on city(city and area) and activity(type)",
)

tools = [
    google_map_tool,
]

def _handle_error(error) -> str:
    print("Error :", error)
    return str(error)

In [68]:
from langchain.agents import tool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.schema import SystemMessage
from langchain.globals import set_debug
set_debug(False)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# llm = ChatOpenAI(openai_api_key=API_KEY, model="gpt-4o")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    max_iterations=3,
    # agent=AgentType.SELF_ASK_WITH_SEARCH,
    handle_parsing_errors=_handle_error,
    verbose=True,
)

r = agent.run(
    """
    Please find 5 hotels in Haeundae, Busan and give me the name, rating of the hotel
    """
)
print("정답은: {}".format(r))



> Entering new AgentExecutor chain...
Thought: I need to find hotels in Haeundae, Busan.
Action: google_map
Action Input: {'activity': 'hotel', 'city': '부산', 'area': 'Haeundae'}
Observation: [{'name': '시티호텔', 'address': '대한민국 부산광역시 동구 중앙대로 294', 'rating': 4.4}, {'name': '부산시티호텔', 'address': '대한민국 부산광역시 연제구 신촌로 19', 'rating': 4.4}, {'name': '한화리조트 해운대', 'address': '대한민국 부산광역시 해운대구 마린시티3로 52', 'rating': 4.2}, {'name': '파크 하얏트 부산', 'address': '대한민국 부산광역시 해운대구 마린시티1로 51', 'rating': 4.6}, {'name': '신라스테이 해운대', 'address': '대한민국 부산광역시 해운대구 해운대로570번길 46', 'rating': 4.2}, {'name': '베스트인시티호텔 부산', 'address': '대한민국 부산광역시 동구 고관로 7', 'rating': 3.9}, {'name': '아르반시티호텔', 'address': '대한민국 부산광역시 연제구 반송로 20', 'rating': 4.1}, {'name': '해운대센텀호텔', 'address': '대한민국 부산광역시 해운대구 센텀3로 20', 'rating': 4}, {'name': '호텔 아벤트리 부산', 'address': '대한민국 부산광역시 중구 광복로39번길 6', 'rating': 4.3}, {'name': 'Urbanstay Busan City hall 어반스테이 부산 시청', 'address': '대한민국 부산광역시 연제구 중앙대로 1116-8 어반스테이 더 시티', 'rating': 4.1}, {'name': '하운드호텔